In [1]:
from transformers import pipeline

/Users/ayushjain/Development/DisruptionLab/jinship_sp2024/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ayushjain/Development/DisruptionLab/jinship_sp2024/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#model for question answering 
pipe = pipeline("question-answering", model="deepset/roberta-base-squad2")

config.json: 100%|██████████| 571/571 [00:00<00:00, 484kB/s]
model.safetensors: 100%|██████████| 496M/496M [00:22<00:00, 21.9MB/s] 
tokenizer_config.json: 100%|██████████| 79.0/79.0 [00:00<00:00, 58.3kB/s]
vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 5.71MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 6.20MB/s]
special_tokens_map.json: 100%|██████████| 772/772 [00:00<00:00, 2.61MB/s]


In [19]:
# reading in csvs
import csv
import chromadb

title_ids = []
genres = []
description = []

#read in genres and descriptions automagically
with open('shows.csv', 'r') as file:
  reader = csv.DictReader(file)
  #only first 100 items
  for i, row in enumerate(reader):
      if i >= 100:
          break
      genres.append(row['listed_in'])
      description.append(row['description'])
      title_ids.append(row['show_id'])    

In [20]:
chroma_client = chromadb.PersistentClient(path="./")
collection = chroma_client.get_or_create_collection(name="Database");

In [ ]:
collection.add(
  documents=description,
  metadatas = [{'source': genre} for genre in genres],
  ids=title_ids
)

In [23]:
generated_genres = chroma_client.get_or_create_collection(name="generated")

In [ ]:
#gets the query and its distance 
collection.query(query_texts="documentary", n_results=5)
#for each description
    #can run a similarity search on the genre database to see if something is above the similarity threshold, if so then that is the value 
    

In [35]:
import uuid
groups = {}

first_desc = description[0]
created_genre = pipe(question="What is a good genre for this description", context=first_desc)

groups[first_desc] = created_genre['answer']

generated_genres.add(
  documents=created_genre,
  metadatas = [{'source':first_desc}],
  ids=[str(uuid.uuid4())]
)

for desc in description[1:]:
  query = generated_genres.query(query_texts=desc, n_results=1)
  distance = query['distances'][0][0]
  genre = query['documents'][0][0]

  if distance < 0.5:
    groups[desc] = genre
  else:
    #first query the text with genres, if it does not have a distance lower than .5 then query the text
    created_genre = pipe(question="What is a good genre for this description", context=desc)['answer']
    #then query the returend genre, if     that does not have a distance lower than .5 than add it ot the genereated_genres
    query = generated_genres.query(query_texts=created_genre, n_results=1)
    distance = query['distances'][0]
    genre = query['documents'][0]
    
    if distance < 0.5:
      groups[desc] = genre
    else:
      groups[desc] = created_genre
      generated_genres.add(
        documents=created_genre,
        metadatas = [{'source':desc}],
        ids=[str(uuid.uuid4())]
      )

print(groups)

ValueError: Number of documents 4 must match number of ids 1